In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.feature_selection import RFE
from sklearn.svm import SVR, LinearSVR
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.preprocessing import normalize, PowerTransformer, power_transform, scale, StandardScaler
from sklearn.pipeline import make_pipeline
from scipy.stats import skew
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import calendar
import os
import geopandas as gpd
from shapely.geometry import Polygon, MultiPolygon
import earthpy as et
from tqdm.notebook import tqdm
from mlxtend.feature_selection import SequentialFeatureSelector as sfs
import json

%matplotlib inline

In [2]:
!wget --directory-prefix=../data/covid/ -Nq https://raw.githubusercontent.com/kaushiktandon/COVID-19-Vaccine-Allocation/master/data/covid/covid_month.csv
!wget --directory-prefix=../data/ -Nq https://raw.githubusercontent.com/kaushiktandon/COVID-19-Vaccine-Allocation/master/data/social_explorer_processed_data.csv
!wget --directory-prefix=../data/ -Nq https://raw.githubusercontent.com/kaushiktandon/COVID-19-Vaccine-Allocation/master/data/social_explorer_processed_data_only_percents.csv
!wget -Nq https://raw.githubusercontent.com/kaushiktandon/COVID-19-Vaccine-Allocation/master/shape/la_shape.zip
!unzip -nq la_shape -d shape

# LA County Shapefile

In [3]:
# Graph neighbors of a region
def graph_neighbors(df2, region):
    df2['plot_col'] = 0
    df2.plot_col = df2.plot_col.mask(df2['region'] == region, 1)
    temp_n = df2[df2.region == region].neighbors.values[0].split(',')
    temp_n = [n.strip() for n in temp_n]
    for n in temp_n:
        df2.plot_col = df2.plot_col.mask(df2['region'] == n, 2)
    ax = df2.plot(column='plot_col', figsize=(10,10))
    green_patch = mpatches.Patch(color='green', label='Covering Polygon')
    yellow_patch = mpatches.Patch(color='yellow', label='Bordering Regions')
    purple_patch = mpatches.Patch(color='purple', label='Inner LA County')
    plt.legend(handles=[green_patch, yellow_patch, purple_patch])
#     ax.annotate()
#     df2.apply(lambda x: ax.annotate(s=x.region, xy=x.geometry.centroid.coords[0], ha='center'),axis=1);
    
# Graph region
def graph_region(df2, region):
    df2['plot_col'] = 0
    df2.plot_col = df2.plot_col.mask(df2['region'] == region, 1)

    df2.plot(column='plot_col', figsize=(10,10), alpha=0.5)
    
def remove_missing_neighbors(df2):
    df2 = df.copy(deep=True)
    df2["neighbors"] = None  # add NEIGHBORS column
    df2["plot_col"] = 0 # Column used to graph neighbors
    # Create a polygon that covers the entire la county region
    cover = gpd.GeoSeries(Polygon([(-119.4,35), (-119.4,33.2), (-117,33.2), (-117,35)])).set_crs(epsg=4269)
    # Remove Catalina Island since it is an island which has no neighbors
    la_noisland = df2[df2['region']!='Unincorporated Catalina Island']
    # Remove small holes within the la county unary union polygon
    no_holes = MultiPolygon(Polygon(p.exterior) for p in la_noisland.geometry.unary_union)
    no_holes_series = gpd.GeoSeries(no_holes).set_crs(epsg=4269)
    # Subtract la county from the covering polygon
    diff = gpd.GeoSeries.difference(cover, no_holes_series)
    # We can find the cities that are touching 'diff' to locate border cities
    # diff.plot(figsize=(5,5))
    # Get neighbors of diff to obtain bordered regions
    border_regions = df2[~df2.geometry.disjoint(diff[0])].region.tolist()
    
    df2.loc[len(df2)] = ["COVER", diff[0], ", ".join(border_regions), 0]
    graph_neighbors(df2, 'COVER')
    df2 = df2[:-1]

In [4]:
# la_shapes = gpd.read_file("shape/la_shape.shp") # L.A. County neighborhoods shapefile
la_shapes = gpd.read_file("shape/la_shape.shp") # L.A. County neighborhoods shapefile
la_filtered = la_shapes.rename(columns={'name':'region'})
la_filtered["plot_col"] = 0 # Column used to graph neighbors
la_filtered = la_filtered[['region','geometry','plot_col']]
la_filtered.head()

,region,geometry,plot_col
0,Acton,"POLYGON ((-118.20262 34.53899, -118.18947 34.5...",0
1,Adams-Normandie,"POLYGON ((-118.30901 34.03741, -118.30041 34.0...",0
2,Agoura Hills,"POLYGON ((-118.76193 34.16820, -118.72632 34.1...",0
3,Agua Dulce,"POLYGON ((-118.25468 34.55830, -118.25551 34.5...",0
4,Alhambra,"POLYGON ((-118.12175 34.10504, -118.11687 34.1...",0


## Smooth/process COVID data

In [5]:
!wget --directory-prefix=../data/covid -Nq https://raw.githubusercontent.com/ANRGUSC/lacounty_covid19_data/master/data/Covid-19.csv
covid_filename = '../data/covid/Covid-19.csv'

covid_df = pd.read_csv(covid_filename)
covid_df = covid_df.rename(columns={'Region':'region', 'Number of cases':'cases'})
covid_df.head()

,Time Stamp,region,Latitude,Longitude,cases
0,03-16-2020,Alhambra,34.093042,-118.127060,2
1,03-16-2020,Arcadia,34.136208,-118.040150,1
2,03-16-2020,Beverly Hills,34.069650,-118.396306,1
3,03-16-2020,Boyle Heights,34.043689,-118.209768,5
4,03-16-2020,Carson,33.832204,-118.251755,1


In [6]:
# Set COVID cases to not be less than previously recorded - it should be a non-decreasing function
for region in covid_df["region"].unique():
  df = covid_df[covid_df['region'] == region]
  prev_cases = 0
  for idx, row in df.iterrows():
    if prev_cases > row['cases']:
      covid_df.loc[idx, 'cases'] =  prev_cases
    else:
      prev_cases = row['cases']

In [7]:
for region in covid_df["region"].unique():
  df = covid_df[covid_df['region'] == region]
  prev_cases = 0
  for idx, row in df.iterrows():
    covid_df.loc[idx, 'active_cases'] =  row['cases'] - prev_cases
    prev_cases = row['cases']
covid_df[covid_df['region'] == 'Alhambra']

,Time Stamp,region,Latitude,Longitude,cases,active_cases
0,03-16-2020,Alhambra,34.093042,-118.12706,2,2.0
31,03-17-2020,Alhambra,34.093042,-118.12706,2,0.0
73,03-18-2020,Alhambra,34.093042,-118.12706,2,0.0
118,03-19-2020,Alhambra,34.093042,-118.12706,2,0.0
188,03-20-2020,Alhambra,34.093042,-118.12706,3,1.0
...,...,...,...,...,...,...
54088,11-4-2020,Alhambra,34.093042,-118.12706,1467,9.0
54326,11-5-2020,Alhambra,34.093042,-118.12706,1472,5.0
54564,11-6-2020,Alhambra,34.093042,-118.12706,1480,8.0
54802,11-7-2020,Alhambra,34.093042,-118.12706,1492,12.0


## Fix inconsistencies in region name mapping

In [8]:
la_to_covid = {
    'Downtown': ['City', 'Pico', 'Wholesale District', 'Central', 'Little Tokyo', 'Temple-Beaudry'],
    'Silver Lake': ['Silverlake'],
    'Vernon': ['West Vernon', 'Vernon Central'],
    'Baldwin Hills/Crenshaw': ['Baldwin Hills'],
    'West Hollywood': ['Park LaBrea'],
    'Mid-Wilshire': ['Park LaBrea', 'Cloverdale/Cochran'],
    'Santa Clarity': ['Canyon Country'],
    'Echo Park': ['Angelino Heights'],
    'Avocado Heights': ['Bassett'],
    'Bel-Air': ['Bel Air'],
    'Arlington Heights': ['Country Club Park'],
    'Exposition Park': ['Exposition'],
    'Chinatown': ['Figueroa Park Square'],
    'Torrance': ['Gramercy Place'],
    'Harbor City': ['Harbor Pines'],
    'Mid-City': ['Lafayette Square', 'Mid-city', 'Reynier Village', 'Victoria Park', 'Wellington Square'],
    'Azusa': ['Lakeview Terrace'],
    'East Hollywood': ['Little Armenia'],
    'Playa Vista': ['Longwood'],
    'Brentwood': ['Mandeville Canyon'],
    'Playa del Rey': ['Playa Del Rey'],
    'Porter Ranch': ['Reseda Ranch'],
    'Carthay': ['South Carthay'],
    'Los Feliz': ['Thai Town'],
    'Toluca Lake': ['Toluca Terrace'],
    'El Sereno': ['University Hills'],
    'Angeles Crest': ['Angeles National Forest'],
    'Mount Washington': ['Mt. Washington']
}

for key, value in la_to_covid.items():
    covid_df.loc[covid_df['region'].isin(value), 'region'] = key

## Month aggregate

In [9]:
covid_df_month = covid_df[['Time Stamp', 'region', 'active_cases']]
covid_df_month['Time Stamp'] = pd.to_datetime(covid_df['Time Stamp'])
covid_df_month = covid_df_month.set_index('Time Stamp')
covid_df_month = covid_df_month.groupby([pd.Grouper(freq='M'), 'region']).sum()
covid_df_month = covid_df_month.reset_index(level=['Time Stamp','region']).rename(columns={'Time Stamp': 'month'})
covid_df_month.month = covid_df_month.month.dt.month
covid_df_month = covid_df_month[covid_df_month.month != 3] ## Remove march data since its not complete
covid_df_month

<ipython-input-9-787227d294ba>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  covid_df_month['Time Stamp'] = pd.to_datetime(covid_df['Time Stamp'])


,month,region,active_cases
219,4,Acton,24.0
220,4,Adams-Normandie,21.0
221,4,Agoura Hills,13.0
222,4,Alhambra,31.0
223,4,Alsace,29.0
...,...,...,...
1940,11,Willowbrook,0.0
1941,11,Wilmington,63.0
1942,11,Wilshire Center,63.0
1943,11,Winnetka,122.0


### Remove regions w/ missing data

In [10]:
covid_unique_regions = pd.DataFrame(data=pd.unique(covid_df.region), columns=['region'])
check_df = la_filtered.assign(InDf2=la_filtered.region.isin(covid_unique_regions.region))
miss_arr = pd.unique(check_df[check_df['InDf2']==False]['region'])
la_processed = la_filtered[~la_filtered['region'].isin(miss_arr)]
la_processed.shape

(195, 3)

## Census data

In [11]:
census_data = pd.read_csv('../data/social_explorer_processed_data_only_percents.csv')
census_unique_regions = pd.DataFrame(data=pd.unique(census_data.name),columns=['region'])
census_data.shape

(261, 168)

## Migration data

In [12]:
migration_data = pd.read_csv('../data/safegraph_aggregated.csv')
migration_data.head(5)

,name,month,Number of incoming visits,Number of outgoing visits (no regI to regI),Number of outgoing visits (with regI to regI),to Acton,to Adams-Normandie,to Agoura Hills,to Agua Dulce,to Alhambra,...,to West San Dimas,to West Whittier-Los Nietos,to Westwood,to Whittier,to Whittier Narrows,to Willowbrook,to Wilmington,to Windsor Square,to Winnetka,to Woodland Hills
0,Agoura Hills,3,21,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Arleta,3,396,16377,16407,0,0,0,0,0,...,0,0,0,0,0,0,0,0,113,237
2,Arlington Heights,3,80,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Atwater Village,3,18134,1334,2116,0,0,0,0,0,...,0,0,0,0,0,0,0,0,8,8
4,Baldwin Hills/Crenshaw,3,316,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [13]:
migration_data = migration_data[['name','month','Number of incoming visits', 'Number of outgoing visits (no regI to regI)']]
migration_data = migration_data.rename(columns={'name':'region','Number of incoming visits':'incoming','Number of outgoing visits (no regI to regI)':'outgoing'})
migration_unique_regions = pd.DataFrame(data=pd.unique(migration_data.region),columns=['region'])

In [14]:
migration_unique_regions

,region
0,Agoura Hills
1,Arleta
2,Arlington Heights
3,Atwater Village
4,Baldwin Hills/Crenshaw
...,...
107,Elysian Park
108,Hansen Dam
109,Adams-Normandie
110,Westwood


## Employment data

In [15]:
employment_data = pd.read_csv('../data/lodes_processed_data.csv')
employment_data = employment_data[['name', 'C000', 'CNS16', 'CNS18']].rename(columns={'name':'region','C000':'total','CNS16':'health','CNS18':'food'})
employment_unique_regions = pd.DataFrame(data=pd.unique(employment_data.region),columns=['region'])
employment_data.head(5)

,region,total,health,food
0,Acton,4021,479,315
1,Adams-Normandie,6100,1017,812
2,Agoura Hills,9539,1058,799
3,Agua Dulce,2158,242,144
4,Alhambra,38542,7022,4191


## Nursing Home data

In [16]:
nursing_data = pd.read_csv('../data/Nursing_Homes_Processed_Data.csv')
nursing_data.head(5)

,Neighborhood,NursingHomeBeds,% of NursingHomeBeds
0,Acton,0,0.000000
1,Adams-Normandie,90,0.117246
2,Agoura Hills,185,0.241005
3,Agua Dulce,0,0.000000
4,Alhambra,1105,1.439514


In [17]:
nursing_data = nursing_data.rename(columns={'Neighborhood':'region','% of NursingHomeBeds':'% beds'})
nursing_unique_regions = pd.DataFrame(data=pd.unique(nursing_data.region),columns=['region'])

In [18]:
# Get all regions that have data
regions = pd.merge(covid_unique_regions, la_processed[['region']], on='region', how='inner')
regions = pd.merge(regions, census_unique_regions, on='region', how='inner')
regions = pd.merge(regions, nursing_unique_regions, on='region', how='inner')
regions = pd.merge(regions, employment_unique_regions, on='region', how='inner')
regions = pd.merge(regions, migration_unique_regions, on='region', how='inner')['region'].values
# print(regions.shape)
len(regions)

91

In [19]:
used_features = [
  'name',
  '% Area Total: Area (Land)',
  '% Area Total: Area (Water)',
  '% Total Population: Under 5 Years',
  '% Total Population: 5 to 9 Years',
  '% Total Population: 10 to 14 Years',
  '% Total Population: 15 to 17 Years',
  '% Total Population: 18 to 24 Years',
  '% Total Population: 25 to 34 Years',
  '% Total Population: 35 to 44 Years',
  '% Total Population: 45 to 54 Years',
  '% Total Population: 55 to 64 Years',
  '% Total Population: 65 to 74 Years',
  '% Total Population: 75 to 84 Years',
  '% Total Population: 85 Years and Over',
  '% Households: Family Households',
  '% Households: Family Households: Married-Couple Family',
  '% Households: Family Households: Other Family',
  '% Households: Family Households: Other Family: Male Householder, No Wife Present',
  '% Households: Family Households: Other Family: Female Householder, No Husband Present',
  '% Households: Nonfamily Households',
  '% Households: Nonfamily Households: Male Householder',
  '% Households: Nonfamily Households: Female Householder',
  '% Population 16 Years and Over: in Labor Force',
  '% Population 16 Years and Over: in Labor Force: in Armed Forces',
  '% Population 16 Years and Over: in Labor Force: Civilian',
  '% Population 16 Years and Over: in Labor Force: Civilian: Employed',
  '% Population 16 Years and Over: in Labor Force: Civilian: Unemployed',
  '% Population 16 Years and Over: Not in Labor Force',
  '% In Labor Force 16 Years and Over: in Armed Forces',
  '% In Labor Force 16 Years and Over: Civilian',
  '% Employed Civilian Population 16 Years and Over: Agriculture, Forestry, Fishing and Hunting, and Mining',
  '% Employed Civilian Population 16 Years and Over: Construction',
  '% Employed Civilian Population 16 Years and Over: Manufacturing',
  '% Employed Civilian Population 16 Years and Over: Wholesale Trade',
  '% Employed Civilian Population 16 Years and Over: Retail Trade',
  '% Employed Civilian Population 16 Years and Over: Transportation and Warehousing, and Utilities',
  '% Employed Civilian Population 16 Years and Over: Information',
  '% Employed Civilian Population 16 Years and Over: Finance and Insurance, and Real Estate and Rental  and Leasing',
  '% Employed Civilian Population 16 Years and Over: Professional, Scientific, and Management, and  Administrative and Waste Management Services',
  '% Employed Civilian Population 16 Years and Over: Educational Services, and Health Care and Social  Assistance',
  '% Employed Civilian Population 16 Years and Over: Arts, Entertainment, and Recreation, and  Accommodation and Food Services',
  '% Employed Civilian Population 16 Years and Over: Other Services, Except Public Administration',
  '% Employed Civilian Population 16 Years and Over: Public Administration',
  '% Employed Civilian Population 16 Years and Over1: Private Sector',
  '% Employed Civilian Population 16 Years and Over1: Public Sector',
  '% Employed Civilian Population 16 Years and Over1: Self-Employed (Incorporated and Not Incorporated)',
  '% Employed Civilian Population 16 Years and Over1: Private Non-Profit',
  '% Employed Civilian Population 16 Years and Over1: Unpaid Family Workers',
  '% Households1: with Earnings', '% Households1: No Earnings',
  '% Households2: with Wage or Salary Income',
  '% Households2: No Wage or Salary Income',
  '% Households3: with Self-Employment Income',
  '% Households3: No Self-Employment Income',
  '% Households4: with Interest, Dividends, or Net Rental Income',
  '% Households4: No Interest, Dividends, or Net Rental Income',
  '% Households5: with Social Security Income',
  '% Households5: No Social Security Income',
  '% Households6: with Supplemental Security Income (Ssi)',
  '% Households6: No Supplemental Security Income (Ssi)',
  '% Households7: with Public Assistance Income',
  '% Households7: No Public Assistance Income',
  '% Households8: with Retirement Income',
  '% Households8: No Retirement Income',
  '% Households9: with Other Types of Income',
  '% Households9: No Other Types of Income',
  '% Families: Income Below Poverty Level',
  '% Families: Income Below Poverty Level: Married Couple Family: with Related Child Living  Bellow Poverty Level',
  '% Families: Income Below Poverty Level: Married Couple Family: No Related Children Under 18 Years',
  '% Families: Income Below Poverty Level: Male Householder, No Wife Present',
  '% Families: Income Below Poverty Level: Male Householder, No Wife Present: with Related Children Under 18 Years',
  '% Families: Income Below Poverty Level: Male Householder, No Wife Present: No Related Children Under 18 Years',
  '% Families: Income Below Poverty Level: Female Householder, No Husband Present',
  '% Families: Income Below Poverty Level: Female Householder, No Husband Present: with Related Children Under 18 Years',
  '% Families: Income Below Poverty Level: Female Householder, No Husband Present: No Related Children Under 18 Years',
  '% Families: Income At or Above Poverty Level',
  '% Population Under 18 Years of Age for Whom Poverty Status Is Determined: Living in Poverty',
  '% Population Under 18 Years of Age for Whom Poverty Status Is Determined: At or Above Poverty Level',
  '% Population Age 18 to 64 for Whom Poverty Status  Is Determined: Living in Poverty',
  '% Population Age 18 to 64 for Whom Poverty Status  Is Determined: At or Above Poverty Level',
  '% Population Age 65 and Over for Whom Poverty  Status Is Determined: Living in Poverty',
  '% Population Age 65 and Over for Whom Poverty  Status Is Determined: At or Above Poverty Level',
  '% Population for Whom Poverty Status Is Determined: Under .50',
  '% Population for Whom Poverty Status Is Determined: .50 to .74',
  '% Population for Whom Poverty Status Is Determined: .75 to .99',
  '% Population for Whom Poverty Status Is Determined: 1.00 to 1.49',
  '% Population for Whom Poverty Status Is Determined: 1.50 to 1.99',
  '% Population for Whom Poverty Status Is Determined: 2.00 and Over',
  '% Population for Whom Poverty Status Is Determined1: Under 1.00 (Doing Poorly)',
  '% Population for Whom Poverty Status Is Determined1: 1.00 to 1.99 (Struggling)',
  '% Population for Whom Poverty Status Is Determined1: Under 2.00 (Poor or Struggling)',
  '% Population for Whom Poverty Status Is Determined1: 2.00 and Over (Doing Ok)',
  '% White Alone Population for Whom Poverty Status Is  Determined: Income Below Poverty Level',
  '% White Alone Population for Whom Poverty Status Is  Determined: Income At or Above Poverty Level',
  '% Black or African American Alone Population for  Whom&nbsp; Poverty Status Is Determined: Income Below Poverty Level',
  '% Black or African American Alone Population for  Whom&nbsp; Poverty Status Is Determined: Income At or Above Poverty Level',
  '% American Indian and Alaska Native Alone  Population For&nbsp; Whom Poverty Status Is Determined: Income Below Poverty Level',
  '% American Indian and Alaska Native Alone  Population For&nbsp; Whom Poverty Status Is Determined: Income At or Above Poverty Level',
  '% Asian Alone Population for Whom Poverty Status Is  Determined: Income Below Poverty Level',
  '% Asian Alone Population for Whom Poverty Status Is  Determined: Income At or Above Poverty Level',
  '% Native Hawaiian and Other Pacific Islander Alone &nbsp; Population for Whom Poverty Status Is Determined: Income Below Poverty Level',
  '% Native Hawaiian and Other Pacific Islander Alone &nbsp; Population for Whom Poverty Status Is Determined: Income At or Above Poverty Level',
  '% Some Other Race Alone Population for Whom Poverty  Status Is Determined: Income Below Poverty Level',
  '% Some Other Race Alone Population for Whom Poverty  Status Is Determined: Income At or Above Poverty Level',
  '% Two or More Races Population for Whom Poverty  Status Is Determined: Income Below Poverty Level',
  '% Two or More Races Population for Whom Poverty  Status Is Determined: Income At or Above Poverty Level',
  '% Hispanic or Latino Population for Whom Poverty  Status Is Determined: Income Below Poverty Level',
  '% Hispanic or Latino Population for Whom Poverty  Status Is Determined: Income At or Above Poverty Level',
  '% White Alone, Not Hispanic or Latino Population  for Whom&nbsp; Poverty Status Is Determined: Income Below Poverty Level',
  '% White Alone, Not Hispanic or Latino Population  for Whom&nbsp; Poverty Status Is Determined: Income At or Above Poverty Level',
  '% Workers 16 Years and Over: Car, Truck, or Van',
  '% Workers 16 Years and Over: Drove Alone',
  '% Workers 16 Years and Over: Carpooled',
  '% Workers 16 Years and Over: Public Transportation (Includes Taxicab)',
  '% Workers 16 Years and Over: Motorcycle',
  '% Workers 16 Years and Over: Bicycle',
  '% Workers 16 Years and Over: Walked',
  '% Workers 16 Years and Over: Other Means',
  '% Workers 16 Years and Over: Worked At Home',
  '% Workers 16 Years and Over in Households: Householder Lived in Renter-Occupied Housing Units',
  '% Workers 16 Years and Over in Households: Householder Lived in Renter-Occupied Housing Units: Car, Truck, or Van - Drove Alone',
  '% Workers 16 Years and Over in Households: Householder Lived in Renter-Occupied Housing Units: Car, Truck, or Van - Carpooled',
  '% Workers 16 Years and Over in Households: Householder Lived in Renter-Occupied Housing Units: Public Transportation (Excluding Taxicab)',
  '% Workers 16 Years and Over in Households: Householder Lived in Renter-Occupied Housing Units: Walked',
  '% Workers 16 Years and Over in Households: Householder Lived in Renter-Occupied Housing Units: Taxicab, Motorcycle, Bicycle, or Other Means',
  '% Workers 16 Years and Over in Households: Householder Lived in Renter-Occupied Housing Units: Worked At Home',
  '% Workers 16 Years and Over2: Did Not Work At Home',
  '% Workers 16 Years and Over2: Did Not Work At Home: Less than 10 Minutes',
  '% Workers 16 Years and Over2: Did Not Work At Home: 10 to 19 Minutes',
  '% Workers 16 Years and Over2: Did Not Work At Home: 20 to 29 Minutes',
  '% Workers 16 Years and Over2: Did Not Work At Home: 30 to 39 Minutes',
  '% Workers 16 Years and Over2: Did Not Work At Home: 40 to 59 Minutes',
  '% Workers 16 Years and Over2: Did Not Work At Home: 60 to 89 Minutes',
  '% Workers 16 Years and Over2: Did Not Work At Home: 90 or More Minutes',
  '% Workers 16 Years and Over2: Worked At Home',
  '% Total: No Health Insurance Coverage',
  '% Total: with Health Insurance Coverage',
  '% Total: with Health Insurance Coverage: Public Health Coverage',
  '% Total: with Health Insurance Coverage: Private Health Insurance',
  '% Population Under 18: No Health Insurance Coverage',
  '% Population Under 18: with Health Insurance Coverage',
  '% Population Under 18: with Health Insurance Coverage: Public Health Coverage',
  '% Population Under 18: with Health Insurance Coverage: Private Health Insurance',
  '% Population 18 to 24: No Health Insurance Coverage',
  '% Population 18 to 24: with Health Insurance Coverage',
  '% Population 18 to 24: with Health Insurance Coverage: Public Health Coverage',
  '% Population 18 to 24: with Health Insurance Coverage: Private Health Insurance',
  '% Population 25 to 34: No Health Insurance Coverage',
  '% Population 25 to 34: with Health Insurance Coverage',
  '% Population 25 to 34: with Health Insurance Coverage: Public Health Coverage',
  '% Population 25 to 34: with Health Insurance Coverage: Private Health Insurance',
  '% Population 35 to 64: No Health Insurance Coverage',
  '% Population 35 to 64: with Health Insurance Coverage',
  '% Population 35 to 64: with Health Insurance Coverage: Public Health Coverage',
  '% Population 35 to 64: with Health Insurance Coverage: Private Health Insurance',
  '% Population 65 or Older: No Health Insurance Coverage',
  '% Population 65 or Older: with Health Insurance Coverage',
  '% Population 65 or Older: with Health Insurance Coverage: Public Health Coverage',
  '% Population 65 or Older: with Health Insurance Coverage: Private Health Insurance',
  '% Own Children under 18 Years: Children Living with Single Parents',
  '% Households10: 1-Person Household',
  '% Households10: 2-Person Household',
  '% Households10: 3-Person Household',
  '% Households10: 4-Person Household',
  '% Households10: 5-Person Household',
  '% Households10: 6-Person Household',
  '% Households10: 7-or-More Person Household'
]

for feature in census_data.columns:
    if feature not in used_features:
        del census_data[feature]
        print("Deleted ", feature)

census_data.head()
feature_names = np.concatenate((['cases', 'County total cases', 'Incoming migration','Outgoing migration','Nursing home density', 'Total jobs', 'Healthcare workers', 'Food Services workers'], census_data.columns.values[1:]))
feature_names

array(['cases', 'County total cases', 'Incoming migration',
       'Outgoing migration', 'Nursing home density', 'Total jobs',
       'Healthcare workers', 'Food Services workers',
       '% Area Total: Area (Land)', '% Area Total: Area (Water)',
       '% Total Population: Under 5 Years',
       '% Total Population: 5 to 9 Years',
       '% Total Population: 10 to 14 Years',
       '% Total Population: 15 to 17 Years',
       '% Total Population: 18 to 24 Years',
       '% Total Population: 25 to 34 Years',
       '% Total Population: 35 to 44 Years',
       '% Total Population: 45 to 54 Years',
       '% Total Population: 55 to 64 Years',
       '% Total Population: 65 to 74 Years',
       '% Total Population: 75 to 84 Years',
       '% Total Population: 85 Years and Over',
       '% Households: Family Households',
       '% Households: Family Households: Married-Couple Family',
       '% Households: Family Households: Other Family',
       '% Households: Family Households: Other Fam

In [20]:
def build_month_data_with_census(census_data):
    month_data_with_census = np.array([])
    # List of months
    time_frames = pd.unique(covid_df_month.month)
    # List of month case sums
    covid_month_gp = covid_df_month.groupby('month').sum().reset_index()

    for i in range(len(time_frames)-2): ## Need to ignore last month since data for last month is not complete
        time_frame = time_frames[i]
        for region in regions:
            # Build current row of data
            row_data = np.array([time_frame])
            # Get cases for current region
            region_cases = covid_df_month.loc[(covid_df_month.month == time_frame) & (covid_df_month.region == region)]['active_cases'].values
            if len(region_cases) == 0:
                region_cases = 0
            else:
                region_cases = region_cases[0]
            row_data = np.append(row_data, [region_cases])

            # Get cases of the entire LA region
            total_cases = covid_month_gp[covid_month_gp['month'] == time_frame]['active_cases'].values[0]
            row_data = np.append(row_data, [total_cases])
            
            # Incoming migration
            in_mig = migration_data[(migration_data['month'] == time_frame) & (migration_data['region'] == region)]['incoming'].values
            if len(in_mig) == 0:
                in_mig = 0
            else:
                in_mig = in_mig[0]
            row_data = np.append(row_data, [in_mig])

            # Outgoing migration
            out_mig = migration_data[(migration_data['month'] == time_frame) & (migration_data['region'] == region)]['outgoing'].values
            if len(out_mig) == 0:
                out_mig = 0
            else:
                out_mig = out_mig[0]
            row_data = np.append(row_data, [out_mig])
            
            # Nusring home
            nurse_den = nursing_data[nursing_data['region'] == region]['% beds'].values
            if len(nurse_den) == 0:
                nurse_den = 0
            else:
                nurse_den = nurse_den[0]
            row_data = np.append(row_data, [nurse_den])
            
            # Total jobs
            emp_data = employment_data[employment_data['region'] == region].values[0]
            row_data = np.append(row_data, [emp_data[1], emp_data[2], emp_data[3]])

            # Get census data
            census_row = census_data[census_data['name']==region].values[0][1:]/100
            row_data = np.append(row_data, census_row)

            # Get cases for next time_frame
            next_cases = covid_df_month.loc[(covid_df_month.month == time_frames[i+1]) & (covid_df_month.region == region)]['active_cases'].values
            # This data is not available yet
            if len(next_cases) == 0:
                next_cases = 0
            else:
                next_cases = next_cases[0]
            row_data = np.append(row_data, [next_cases])

            # Add data to numpy arr
            if len(month_data_with_census.shape) == 1:
                month_data_with_census = np.array([row_data])
            else:
                month_data_with_census = np.append(month_data_with_census, [row_data], axis=0)
                month_data_with_census = month_data_with_census.astype(float)
                month_data_with_census[:,~np.isnan(month_data_with_census).any(axis=0)].shape
    return month_data_with_census

month_data_with_census = build_month_data_with_census(census_data)
month_data_with_census

array([[4.00000000e+00, 7.10000000e+01, 1.87920000e+04, ...,
        1.83572923e-02, 6.07410410e-04, 3.30000000e+01],
       [4.00000000e+00, 1.90000000e+02, 1.87920000e+04, ...,
        6.49930379e-02, 6.79416824e-02, 5.96000000e+02],
       [4.00000000e+00, 4.06000000e+02, 1.87920000e+04, ...,
        1.09563865e-03, 1.23701138e-03, 1.20400000e+03],
       ...,
       [9.00000000e+00, 4.70000000e+01, 2.68900000e+04, ...,
        1.16839763e-02, 3.93175074e-02, 7.00000000e+01],
       [9.00000000e+00, 3.40000000e+01, 2.68900000e+04, ...,
        7.89447387e-03, 4.89657240e-03, 3.90000000e+01],
       [9.00000000e+00, 1.69000000e+02, 2.68900000e+04, ...,
        8.83993504e-02, 6.58758738e-02, 1.88000000e+02]])

In [21]:
month_data_with_census.shape

(546, 177)

# Model Training

## Time Validation

In [22]:
def transform_data(data):
    pipe = make_pipeline(StandardScaler(with_std=False), PowerTransformer(standardize=True))
    if len(data.shape) == 1:
        transformed_data = pipe.fit_transform(data.reshape(-1,1))
        return (transformed_data.reshape(-1,), pipe)
    transformed_data = pipe.fit_transform(data)
    return (transformed_data, pipe)
def apply_transform(pipe, data):
    if len(data.shape) == 1:
        return pipe.transform(data.reshape(-1,1)).reshape(-1,)
    return pipe.transform(data)
def inverse_transform(pipe, data):
    if len(data.shape) == 1:
        return pipe.inverse_transform(data.reshape(-1,1)).reshape(-1,)
    return pipe.inverse_transform(data)

In [23]:
def n_month_window_split(data, n):
    X_train_arr = []
    X_test_arr = []
    y_train_arr = []
    y_test_arr = []
    ptX_arr = []
    pty_arr = []
    for i in range(int(data[0,0]), int(data[-1,0]-n+1)):
        train_start_index = np.where(data[:,0]==i)[0][0]
        train_end_index = np.where(data[:,0]==i+n)[0][0]
        test_start_index = train_end_index
        test_end_index = np.where(data[:,0]==i+n)[0][-1]+1
        X_train = data[train_start_index:train_end_index, 1:-1]
        X_test = data[test_start_index:test_end_index, 1:-1]
        y_train = data[train_start_index:train_end_index, -1]
        y_test = data[test_start_index:test_end_index, -1]
        
#         print('============')
#         print(train_start_index, test_start_index, test_end_index)
#         print('============')
        
        # Feature transformation
        ## Transformation should be done on the training set and applied on testing set
        X_train, pt_X = transform_data(X_train)
        X_test = apply_transform(pt_X, X_test)
        y_train, pt_y = transform_data(y_train)
        y_test = apply_transform(pt_y, y_test)
        
        X_train_arr.append(X_train)
        X_test_arr.append(X_test)
        y_train_arr.append(y_train)
        y_test_arr.append(y_test)
        ptX_arr.append(pt_X)
        pty_arr.append(pt_y)
        
    return (X_train_arr, X_test_arr, y_train_arr, y_test_arr, ptX_arr, pty_arr)

# Model Training

In [24]:
def train_model(X_train_arr, X_test_arr, y_train_arr, y_test_arr, model, graph_title, n):
    y_pred_arr = []
    mse_arr = []
    r2_arr = []
    mae_arr = []
    avg_arr = []

    for i in range(len(X_train_arr)):
        model.fit(X_train_arr[i], y_train_arr[i])
        y_pred_arr.append(model.predict(X_test_arr[i]))
        mse_arr.append(mean_squared_error(y_pred_arr[i], y_test_arr[i]))
        r2_arr.append(r2_score(y_pred_arr[i], y_test_arr[i]))
        mae_arr.append(mean_absolute_error(y_pred_arr[i], y_test_arr[i]))
        avg_arr.append(np.average(y_test_arr[i]))
    print('MSE: {a} \nr2: {b} \nMAE: {c} \navg: {d}'.format(a=mse_arr, b=r2_arr, c=mae_arr, d=avg_arr))

    if len(avg_arr)==1:
        # Graph
        fig, ax = plt.subplots(figsize=[8,6])
        ax.scatter(y_test_arr[0], y_pred_arr[0])
        ax.plot([y_test_arr[0].min(), y_test_arr[0].max()], [y_test_arr[0].min(), y_test_arr[0].max()], 'k--', lw=4)
        ax.set_xlabel('Measured')
        ax.set_ylabel('Predicted')
        ax.set_title('{graph_title}\nPrediction for month {m}'.format(graph_title=graph_title, m=calendar.month_name[i+5+n]))
        plt.show()
        return
    fig, axs = plt.subplots(len(avg_arr), figsize=[8,12], constrained_layout=True)
    for i in range(len(avg_arr)):
        axs[i].scatter(y_test_arr[i], y_pred_arr[i])
        axs[i].plot([y_test_arr[i].min(), y_test_arr[i].max()], [y_test_arr[i].min(), y_test_arr[i].max()], 'k--', lw=4)
        axs[i].set_xlabel('Measured')
        axs[i].set_ylabel('Predicted')
        axs[i].set_title('Prediction for {m}'.format(m=calendar.month_name[i+5+n]))
    fig.suptitle(graph_title, fontsize=16)
    plt.show()

In [25]:
def search_model(X_train, X_test, y_train, y_test, model, graph_title):
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_pred, y_test)
    r2 = r2_score(y_pred, y_test)
    mae = mean_absolute_error(y_pred, y_test)
    error = {
        'mse': mse,
        'r2': r2,
        'mae': mae
    }
    print('MSE: {a} \nr2: {b} \nMAE: {c} \navg: {d}'.format(a=mse, b=r2, c=mae, d=np.average(y_test)))
    
    # Graph
    fig, ax = plt.subplots(figsize=[8,6])
    ax.scatter(y_test, y_pred)
    ax.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'k--', lw=4)
    ax.set_xlabel('Measured')
    ax.set_ylabel('Predicted')
    ax.set_title(graph_title)
    plt.show()
    print(model.best_estimator_)

# Gradient Boosting

# Feature Selection

In [26]:
wc5m_X_train_arr, wc5m_X_test_arr, wc5m_y_train_arr, wc5m_y_test_arr, wc5m_ptX_arr, wc5m_pty_arr = n_month_window_split(month_data_with_census, 5)

In [441]:
gb_param = {
    'n_estimators': 700,
    'max_depth': 16,
    'min_samples_split': 100,
    'learning_rate': 0.01,
    'loss': 'ls'
}
gb_model = GradientBoostingRegressor(**gb_param)
gb_feat_subsets = f_selection(gb_model,wc5m_X_train_arr[0].shape[1],wc5m_X_train_arr[0],wc5m_y_train_arr[0])


STOPPING EARLY DUE TO KEYBOARD INTERRUPT...

In [442]:
print(gb_feat_subsets)
np.save('../data/gb_feat_subsets.npy',gb_feat_subsets, allow_pickle=True)

{1: {'feature_idx': (5,), 'cv_scores': array([0.71334786, 0.85631655, 0.4407976 , 0.89241409, 0.65262675]), 'avg_score': 0.7111005703977666, 'feature_names': ('5',)}, 2: {'feature_idx': (5, 173), 'cv_scores': array([0.71631205, 0.87347404, 0.46369581, 0.91649605, 0.62930466]), 'avg_score': 0.7198565217811982, 'feature_names': ('5', '173')}, 3: {'feature_idx': (5, 102, 173), 'cv_scores': array([0.72230848, 0.87870934, 0.46211911, 0.91833898, 0.62882895]), 'avg_score': 0.7220609719982137, 'feature_names': ('5', '102', '173')}, 4: {'feature_idx': (5, 49, 102, 173), 'cv_scores': array([0.7160169 , 0.88020998, 0.46427012, 0.91892262, 0.634972  ]), 'avg_score': 0.7228783254526783, 'feature_names': ('5', '49', '102', '173')}, 5: {'feature_idx': (5, 49, 102, 103, 173), 'cv_scores': array([0.7160169 , 0.88020998, 0.46427012, 0.91892262, 0.634972  ]), 'avg_score': 0.7228783254526783, 'feature_names': ('5', '49', '102', '103', '173')}, 6: {'feature_idx': (5, 48, 49, 102, 103, 173), 'cv_scores': a

In [418]:
gb_feat_subsets = np.load('../data/gb_feat_subsets.npy', allow_pickle=True)[()]
gb_feat_subsets

{1: {'feature_idx': (0,),
  'cv_scores': array([0.73347998, 0.8161115 , 0.34517045, 0.61968864, 0.46885929]),
  'avg_score': 0.5966619720458215,
  'feature_names': ('0',)},
 2: {'feature_idx': (0, 1),
  'cv_scores': array([0.76492683, 0.89453549, 0.35222746, 0.8125219 , 0.52418433]),
  'avg_score': 0.6696792015185971,
  'feature_names': ('0', '1')},
 3: {'feature_idx': (0, 1, 172),
  'cv_scores': array([0.78089029, 0.88822498, 0.35164029, 0.84783849, 0.52083289]),
  'avg_score': 0.6778853874877052,
  'feature_names': ('0', '1', '172')},
 4: {'feature_idx': (0, 1, 129, 172),
  'cv_scores': array([0.79210343, 0.89201727, 0.35353407, 0.86131451, 0.51980067]),
  'avg_score': 0.6837539885960784,
  'feature_names': ('0', '1', '129', '172')},
 5: {'feature_idx': (0, 1, 129, 136, 172),
  'cv_scores': array([0.79385839, 0.88837114, 0.36482614, 0.85014582, 0.53451148]),
  'avg_score': 0.6863425929305487,
  'feature_names': ('0', '1', '129', '136', '172')},
 6: {'feature_idx': (0, 1, 105, 129, 13